# Overdamped

In [1]:
import numpy as np
import folie as fl
import matplotlib.pyplot as plt

Let's first simulate some trajectories with various timestep

In [2]:
model_simu = fl.models.OrnsteinUhlenbeck()
model_simu.coefficients = np.array([0.1, 1.2, 2.0])
data_dts = []
list_dts = [1e-3, 5e-3, 1e-2, 5e-2]
for dt in list_dts:
    simulator = fl.Simulator(fl.simulations.ExactStepper(model_simu), dt)
    data_dts.append(simulator.run(5000, np.zeros((25,)), 25))

ValueError: could not broadcast input array from shape (25,25) into shape (25,1)

We can then run the estimation for various likelihood at all timesteps

In [ ]:
fig, axs = plt.subplots(1, len(model_simu.coefficients))

for name, transitioncls in zip(
    ["Exact", "Euler", "Ozaki", "ShojiOzaki", "Elerian", "Kessler", "Drozdov"],
    [
        fl.ExactDensity,
        fl.EulerDensity,
        fl.OzakiDensity,
        fl.ShojiOzakiDensity,
        fl.ElerianDensity,
        fl.KesslerDensity,
        fl.DrozdovDensity,
    ],
):
    model = fl.models.OrnsteinUhlenbeck()
    estimator = fl.LikelihoodEstimator(transitioncls(model))
    coeffs_vals = np.empty((len(data_dts), len(model.coefficients)))
    for n, data in enumerate(data_dts):
        res = estimator.fit_fetch(data)
        coeffs_vals[n, :] = res.coefficients
    for n in range(len(axs)):
        axs[n].plot(list_dts, np.abs(coeffs_vals[:, n] - model_simu.coefficients[n]), "-+", label=name)
    print(coeffs_vals)
for n in range(len(axs)):
    axs[n].legend()
    axs[n].set_yscale("log")
    axs[n].grid()
    axs[n].set_xlabel(r"$\Delta t$")
    axs[n].set_ylabel(r"$|c-c_{real}|$")
plt.show()
